In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from torch.utils.data import WeightedRandomSampler
from PIL import Image
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import torchvision.models as models
from tqdm import tqdm
import copy


In [19]:
def save_to_list(path):
    output_root =path
    X = []
    y = []
    l=[]
        
    label_to_index = {
        'AVM': 0,
        'Normal': 1,
        'Ulcer': 2
    }
    print(label_to_index)
    for root, _, files in os.walk(output_root):
        for file in files:
            if file.endswith(".bmp"): 
                input_path = os.path.join(root, file)

                base, ext = os.path.splitext(file)
                new_filename = f"{base}{ext}"
                image = cv2.imread(input_path)

                X.append(image)
                y.append(label_to_index[new_filename.split("_")[0]])
                l.append(new_filename)
    return X, y, l

In [13]:
# https://discuss.pytorch.org/t/balanced-sampling-between-classes-with-torchvision-dataloader/2703/2

class ImageDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform 
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        image = Image.fromarray(self.images[idx])  
        label = torch.tensor(self.labels[idx], dtype=torch.long) 

        if self.transform:
            image = self.transform(image)
            
        
        return image, label

In [15]:
class ResNet50Transfer(nn.Module):
    def __init__(self, num_classes=3):
        super(ResNet50Transfer, self).__init__()
        self.base_model = models.resnet50(pretrained=True)

        # Freeze all layers (optional: you can unfreeze later for fine-tuning)
        for param in self.base_model.parameters():
            param.requires_grad = False
        
        for name, param in self.base_model.named_parameters():
            if "layer4" in name or "layer3" in name:  # Unfreeze only final block
                param.requires_grad = True

        in_features = self.base_model.fc.in_features  # Usually 2048
        self.base_model.fc = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(512, num_classes) 
        )

    def forward(self, x):
        return self.base_model(x)


In [16]:
def train_and_evaluate_full(model, train_loader, test_loader, save_path, epochs=15, learning_rate=0.001, device='cuda' if torch.cuda.is_available() else 'cpu'):
    print(f"Using device: {device}") 
    model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=0.01)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)

    best_test_loss = float('inf')
    best_model_weights = copy.deepcopy(model.state_dict())

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct, total = 0, 0
        all_labels = []
        all_predictions = []

        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

        train_loss = running_loss / total
        train_acc = correct / total
        train_precision = precision_score(all_labels, all_predictions, average='weighted', zero_division=0)
        train_recall = recall_score(all_labels, all_predictions, average='weighted', zero_division=0)
        train_f1 = f1_score(all_labels, all_predictions, average='weighted', zero_division=0)

        # Evaluation on test set
        model.eval()
        test_loss = 0.0
        test_correct, test_total = 0, 0
        test_labels, test_predictions = [], []

        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)

                test_loss += loss.item() * images.size(0)
                _, predicted = torch.max(outputs, 1)
                test_correct += (predicted == labels).sum().item()
                test_total += labels.size(0)
                test_labels.extend(labels.cpu().numpy())
                test_predictions.extend(predicted.cpu().numpy())

        test_loss /= test_total
        test_acc = test_correct / test_total
        test_precision = precision_score(test_labels, test_predictions, average='weighted', zero_division=0)
        test_recall = recall_score(test_labels, test_predictions, average='weighted', zero_division=0)
        test_f1 = f1_score(test_labels, test_predictions, average='weighted', zero_division=0)

        print(f"Epoch {epoch+1}/{epochs}")
        print(f"Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, F1: {train_f1:.4f}")
        print(f"Test  Loss: {test_loss:.4f}, Acc: {test_acc:.4f}, Precision: {test_precision:.4f}, Recall: {test_recall:.4f}, F1: {test_f1:.4f}")

        scheduler.step()

        # Save best model based on test loss
        if test_loss < best_test_loss:
            best_test_loss = test_loss
            best_model_weights = copy.deepcopy(model.state_dict())
            print("Best model updated!")

    # Load best model for final evaluation
    model.load_state_dict(best_model_weights)
    torch.save(model.state_dict(), save_path)
    print(f"Best model saved as {save_path}'") # rozszerzenie .pth

    # Final evaluation on test set
    model.eval()
    correct, total = 0, 0
    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

    test_acc = correct / total
    test_precision = precision_score(all_labels, all_predictions, average='weighted', zero_division=0)
    test_recall = recall_score(all_labels, all_predictions, average='weighted', zero_division=0)
    test_f1 = f1_score(all_labels, all_predictions, average='weighted', zero_division=0)
    test_conf_matrix = confusion_matrix(all_labels, all_predictions)

    print("\n Final Test Evaluation:")
    print(f"Accuracy: {test_acc:.4f}, Precision: {test_precision:.4f}, Recall: {test_recall:.4f}, F1: {test_f1:.4f}")
    print("Confusion Matrix:\n", test_conf_matrix)

    return model


In [ ]:
X, y, l = save_to_list("output_samples2")

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123, stratify=y)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=123, stratify=y_test)

In [14]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),          
    transforms.RandomRotation((-180,180)),  
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),                  
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])  # do testów - bez zmian


train_dataset = ImageDataset(X_train, y_train, transform=train_transform)
test_dataset = ImageDataset(X_test, y_test, transform=test_transform)

class_counts = np.bincount(y)  
class_weights = 1.0 / class_counts 

sample_weights_tr = [class_weights[label] for label in y_train]
sample_weights_ts = [class_weights[label] for label in y_test]

samplertr = WeightedRandomSampler(sample_weights_tr, num_samples= len(sample_weights_tr), replacement=True)
#samplerts = WeightedRandomSampler(sample_weights_ts, num_samples= len(sample_weights_ts), replacement=True)

batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler = samplertr)

test_loader = DataLoader(test_dataset, batch_size=batch_size)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [17]:
model = ResNet50Transfer(num_classes=3)
trained_model = train_and_evaluate_full(model, train_loader, test_loader, "model_output_samples_25.04.pth")

c:\Users\alapr\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\alapr\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Using device: cuda


Epoch 1/15: 100%|██████████| 20/20 [00:05<00:00,  3.79it/s]


Epoch 1/15
Train Loss: 0.7697, Acc: 0.6453, Precision: 0.6472, Recall: 0.6453, F1: 0.6461
Test  Loss: 26.2028, Acc: 0.5865, Precision: 0.8362, Recall: 0.5865, F1: 0.6081
Best model updated!


Epoch 2/15: 100%|██████████| 20/20 [00:05<00:00,  3.91it/s]


...

Epoch 13/15
Train Loss: 0.1151, Acc: 0.9599, Precision: 0.9600, Recall: 0.9599, F1: 0.9599
Test  Loss: 0.1138, Acc: 0.9624, Precision: 0.9641, Recall: 0.9624, F1: 0.9627
Best model updated!


Epoch 14/15: 100%|██████████| 20/20 [00:05<00:00,  3.69it/s]


Epoch 14/15
Train Loss: 0.0616, Acc: 0.9807, Precision: 0.9807, Recall: 0.9807, F1: 0.9807
Test  Loss: 0.1105, Acc: 0.9549, Precision: 0.9556, Recall: 0.9549, F1: 0.9546
Best model updated!


Epoch 15/15: 100%|██████████| 20/20 [00:05<00:00,  3.69it/s]


Epoch 15/15
Train Loss: 0.0788, Acc: 0.9679, Precision: 0.9679, Recall: 0.9679, F1: 0.9679
Test  Loss: 0.0860, Acc: 0.9624, Precision: 0.9641, Recall: 0.9624, F1: 0.9627
Best model updated!
Best model saved as model_output_samples_25.04.pth'

 Final Test Evaluation:
Accuracy: 0.9624, Precision: 0.9641, Recall: 0.9624, F1: 0.9627
Confusion Matrix:
 [[25  1  0]
 [ 3 83  1]
 [ 0  0 20]]


Do porównania model wytrenowany na zdjęciach z folderu output.

In [20]:
X, y, l = save_to_list("output")

{'AVM': 0, 'Normal': 1, 'Ulcer': 2}


In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123, stratify=y)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=123, stratify=y_test)

In [22]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),          
    transforms.RandomRotation((-180,180)),  
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),                  
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])  # do testów - bez zmian


train_dataset = ImageDataset(X_train, y_train, transform=train_transform)
test_dataset = ImageDataset(X_test, y_test, transform=test_transform)

class_counts = np.bincount(y)  
class_weights = 1.0 / class_counts 

sample_weights_tr = [class_weights[label] for label in y_train]
sample_weights_ts = [class_weights[label] for label in y_test]

samplertr = WeightedRandomSampler(sample_weights_tr, num_samples= len(sample_weights_tr), replacement=True)
#samplerts = WeightedRandomSampler(sample_weights_ts, num_samples= len(sample_weights_ts), replacement=True)

batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler = samplertr)

test_loader = DataLoader(test_dataset, batch_size=batch_size)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [23]:
model = ResNet50Transfer(num_classes=3)
trained_model = train_and_evaluate_full(model, train_loader, test_loader, "model_output_simple.pth")

c:\Users\alapr\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\alapr\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Using device: cuda


Epoch 1/15: 100%|██████████| 72/72 [00:16<00:00,  4.26it/s]


Epoch 1/15
Train Loss: 0.4553, Acc: 0.8330, Precision: 0.8335, Recall: 0.8330, F1: 0.8329
Test  Loss: 0.2959, Acc: 0.8864, Precision: 0.8981, Recall: 0.8864, F1: 0.8888
Best model updated!


Epoch 2/15: 100%|██████████| 72/72 [00:16<00:00,  4.43it/s]


...

Epoch 13/15
Train Loss: 0.0392, Acc: 0.9856, Precision: 0.9856, Recall: 0.9856, F1: 0.9856
Test  Loss: 0.0614, Acc: 0.9817, Precision: 0.9817, Recall: 0.9817, F1: 0.9817
Best model updated!


Epoch 14/15: 100%|██████████| 72/72 [00:16<00:00,  4.48it/s]


Epoch 14/15
Train Loss: 0.0446, Acc: 0.9870, Precision: 0.9870, Recall: 0.9870, F1: 0.9870
Test  Loss: 0.0814, Acc: 0.9777, Precision: 0.9781, Recall: 0.9777, F1: 0.9778


Epoch 15/15: 100%|██████████| 72/72 [00:15<00:00,  4.69it/s]


Epoch 15/15
Train Loss: 0.0438, Acc: 0.9865, Precision: 0.9865, Recall: 0.9865, F1: 0.9865
Test  Loss: 0.0507, Acc: 0.9899, Precision: 0.9900, Recall: 0.9899, F1: 0.9898
Best model updated!
Best model saved as model_output_simple.pth'

 Final Test Evaluation:
Accuracy: 0.9899, Precision: 0.9900, Recall: 0.9899, F1: 0.9898
Confusion Matrix:
 [[ 96   4   0]
 [  0 322   1]
 [  0   0  70]]
